In [42]:
from imblearn.under_sampling import NearMiss
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import numpy as np
data_regression = pd.read_csv("../data/laba3.csv")
data_classification = pd.read_csv("../data/data_cl.csv")
data_classification['Fire Alarm'] = data_classification['Fire Alarm'].map({"Yes": 1, "No": 0})

In [43]:
data_regression.head()

,Unnamed: 0,Year,Distance,Engine_capacity,Price,Make_ARO,Make_Abarth,Make_Acura,Make_Alfa Romeo,Make_Alta marca,...,Style_Sedan,Style_Universal,Fuel_type_Diesel,Fuel_type_Electric,Fuel_type_Hybrid,Fuel_type_Metan/Propan,Fuel_type_Petrol,Fuel_type_Plug-in Hybrid,Transmission_Automatic,Transmission_Manual
0,0,2011.0,195000.0,1800.0,7750.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,1,2014.0,135000.0,1500.0,8550.0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
2,2,1998.0,1.0,1400.0,2200.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,3,2012.0,110000.0,1500.0,6550.0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
4,4,2006.0,200000.0,1600.0,4100.0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [44]:
data_classification.head()

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],Fire Alarm
0,17.557,53.23,1109.0,490.0,12849.0,19453,938.905,1
1,21.330,47.26,132.0,400.0,13216.0,20085,939.646,1
2,12.206,53.22,1153.0,408.0,12878.0,19437,938.757,1
3,26.510,49.45,1289.0,400.0,12960.0,19405,938.688,1
4,19.910,53.67,124.0,400.0,13220.0,20098,939.656,1


In [45]:

y_regression = data_regression["Price"]
X_regression = data_regression.drop(columns = ['Price'])
y_classification = data_classification['Fire Alarm']
X_classification = data_classification.drop(columns = ['Fire Alarm'])
X_regression=X_regression.drop(['Unnamed: 0'],axis=1)


In [46]:
from sklearn.model_selection import train_test_split
X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.25)
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.25)

In [47]:
X_classification

,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa]
0,17.557,53.23,1109.0,490.0,12849.0,19453,938.905
1,21.330,47.26,132.0,400.0,13216.0,20085,939.646
2,12.206,53.22,1153.0,408.0,12878.0,19437,938.757
3,26.510,49.45,1289.0,400.0,12960.0,19405,938.688
4,19.910,53.67,124.0,400.0,13220.0,20098,939.656
...,...,...,...,...,...,...,...
61156,20.640,53.12,218.0,400.0,13151.0,20050,939.615
61157,25.780,49.44,883.0,671.0,12788.0,19508,939.112
61158,20.530,47.10,59.0,400.0,13205.0,20113,939.626
61159,30.070,38.34,31.0,426.0,12862.0,20765,937.568


In [48]:

y_classification

0        1
1        1
2        1
3        1
4        1
        ..
61156    1
61157    1
61158    0
61159    0
61160    1
Name: Fire Alarm, Length: 61161, dtype: int64

# Регрессия :

In [26]:

# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(934,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [27]:
# посмотрим, какая сеть у нас получилась
model_regression.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 64)                59840     
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                                 
 dropout_6 (Dropout)         (None, 16)                0         
                                                                 
 dense_17 (Dense)            (None, 1)                 17        
                                                                 
Total params: 62465 (244.00 KB)
Trainable params: 6246

In [28]:
# компилируем
model_regression.compile(optimizer=tf.keras.optimiz+ers.Adam(learning_rate=0.001), loss="mse")

In [29]:
# обучаем, 10 эпох означает 10 проходов по обучающей выборке
model_regression.fit(X_regression_train, y_regression_train, epochs=10)

Epoch 1/10
689/689 [==============================] - 7s 6ms/step - loss: 30499918.0000
Epoch 2/10
689/689 [==============================] - 4s 5ms/step - loss: 24195750.0000
Epoch 3/10
689/689 [==============================] - 4s 5ms/step - loss: 23523732.0000
Epoch 4/10
689/689 [==============================] - 4s 5ms/step - loss: 22827014.0000
Epoch 5/10
689/689 [==============================] - 4s 5ms/step - loss: 22905764.0000
Epoch 6/10
689/689 [==============================] - 4s 6ms/step - loss: 22579214.0000
Epoch 7/10
689/689 [==============================] - 4s 5ms/step - loss: 22796694.0000
Epoch 8/10
689/689 [==============================] - 4s 5ms/step - loss: 22406522.0000
Epoch 9/10
689/689 [==============================] - 3s 5ms/step - loss: 22473340.0000
Epoch 10/10
689/689 [==============================] - 4s 5ms/step - loss: 22374610.0000


In [30]:
# оцениваем качество с помощью метрик
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))

230/230 [==============================] - 1s 3ms/step
3486.9738600178425
230/230 [==============================] - 1s 4ms/step
20814865.621717647


Для данной задачи применение нейронных сетей менее целесообразно, чем обычная линейная регрессия

# Классификация :

In [54]:
model_cl = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation="tanh", input_shape=(7,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="tanh"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(2, activation="softmax"),
])

model_cl.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss="binary_crossentropy")
model_cl.fit(X_classification_train, y_classification_train, epochs=10)
y_pred_prob = model_cl.predict(X_classification_test)
y_pred = tf.argmax(y_pred_prob, axis=1)
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))


Epoch 1/10
1434/1434 [==============================] - 9s 5ms/step - loss: 0.5023
Epoch 2/10
1434/1434 [==============================] - 7s 5ms/step - loss: 0.5633
Epoch 3/10
1434/1434 [==============================] - 6s 4ms/step - loss: 0.5603
Epoch 4/10
1434/1434 [==============================] - 6s 4ms/step - loss: 0.5523
Epoch 5/10
1434/1434 [==============================] - 6s 4ms/step - loss: 0.5565
Epoch 6/10
1434/1434 [==============================] - 7s 5ms/step - loss: 0.5568
Epoch 7/10
1434/1434 [==============================] - 7s 5ms/step - loss: 0.5609
Epoch 8/10
1434/1434 [==============================] - 6s 4ms/step - loss: 0.5615
Epoch 9/10
1434/1434 [==============================] - 5s 4ms/step - loss: 0.5533
Epoch 10/10
478/478 [==============================] - 2s 3ms/step
              precision    recall  f1-score   support

           0       0.29      1.00      0.44      4370
           1       0.00      0.00      0.00     10921

    accuracy          

C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
w0 = 1 / y_classification_train[y_classification_train==0].shape[0]
w1 = 1 / y_classification_train[y_classification_train==1].shape[0]


In [56]:
total_samples = len(y_classification_train)
class_weight = {0: total_samples / (2 * np.sum(y_classification_train == 0)),
                1: total_samples / (2 * np.sum(y_classification_train == 1))}

In [59]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(32, activation="relu", input_shape=(7,)),
        tf.keras.layers.Dense(16, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(8, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="softmax"),
    ]
)
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")
model_classification_1.fit(X_classification_train, y_classification_train, epochs=10, verbose=None,
                           class_weight={0: w0, 1: w1})
y_pred = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, y_pred))
print(confusion_matrix(y_classification_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4370
           1       0.71      1.00      0.83     10921

    accuracy                           0.71     15291
   macro avg       0.36      0.50      0.42     15291
weighted avg       0.51      0.71      0.60     15291

[[    0  4370]
 [    0 10921]]


C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\djego\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# MLP